In [1]:
import os
from lxml import etree
import re
import csv

In [2]:
# set dtsx package file
xml_file = rf'../xml_files/SSIS_SQLExtractSample.xml'#HR_DataConductor.dtsx'
if not os.path.isfile(xml_file):
    print("no package fible")

In [3]:
# read and parse ssis package
tree = etree.parse(xml_file)
root = tree.getroot()
root

<Element {www.microsoft.com/SqlServer/Dts}Executable at 0x1ed6ad16cc0>

In [4]:
# Define the tags we needs to search the SSIS package for (all unique tags available in ele_tags variable)
pfx = '{www.microsoft.com/'
ref_tag = pfx + 'SqlServer/Dts}refId'
exe_tag = pfx + 'SqlServer/Dts}Executable'
obj_tag = pfx + 'SqlServer/Dts}ObjectName'
dat_tag = pfx + 'SqlServer/Dts}ObjectData'
tsk_tag = pfx + 'sqlserver/dts/tasks/sqltask}SqlTaskData'
src_tag = pfx + 'sqlserver/dts/tasks/sqltask}SqlStatementSource'
desc_tag = pfx + 'SqlServer/Dts}Description'

In [5]:
# Define the namespace
namespaces = {'DTS': 'www.microsoft.com/SqlServer/Dts'}

In [15]:
sql_queries = []
load_tables = []
my_objects = []

In [16]:
# Loop over the found elements and extract the text between the property tags
for id, task in enumerate(root.xpath('//DTS:Executable[@DTS:Description="Data Flow Task"]', namespaces=namespaces)):
    refid = task.get(ref_tag)
    
    for comp_id, comp in enumerate(task.xpath('.//component', namespaces=namespaces)):
            # If the text between property tag is not empty
        task_name = comp.get('name')
        comp_ref_id = comp.get('refId')

        for prop in comp.xpath('.//property', namespaces=namespaces):
            # If prop.text is not null
            if prop.text:

                if comp.get('componentClassID') == 'Microsoft.OLEDBSource' and (prop.get('description') == 'Specifies the name of the database object used to open a rowset.' or prop.get('name') == 'SqlCommand'):# and comp.get('componentClassID') == 'Microsoft.OLEDBSource':
                    sql_query = prop.text.strip()
                    my_objects.append((id, sql_query, comp_ref_id))

                # if prop.get('description') == 'Specifies the name of the database object used to open a rowset.' and comp.get('componentClassID') == 'Microsoft.OLEDBDestination':
                #     load_table = prop.text.strip()
                #     my_objects.append((id, load_table, comp_ref_id))
                                    
            else:
                for prop in comp.xpath('.//connection[@name="OleDbConnection"]', namespaces=namespaces):
                    source_conn = prop.get('connectionManagerID')
                    print(source_conn)
                    
      
            # sql_query = comp.xpath('.//property[@name="SqlCommand"]/text()')
            # load_table = comp.xpath('.//property[@description="Specifies the name of the database object used to open a rowset."]/text()')
       
            # print(id, sql_query, load_table)
        
    

In [17]:
for tables in my_objects:
    print(tables)

(0, 'SELECT *\n                                                            FROM Test.DataFlow;', 'Package\\SEQ_Loader\\HR Data Conductor\\Cost of Absence\\CostOfAbsence_Sickness Table\\Create table for CostOfAbsence_Sickness')
(0, '[load].[CostOfAbsence_Sickness]', 'Package\\SEQ_Loader\\HR Data Conductor\\Cost of Absence\\CostOfAbsence_Sickness Table\\load CostOfAbsence_Sickness')


In [ ]:
# Write the file names and descriptions to a CSV file with the specified structure
with open('HR_DataConductor_Load_Tables.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['ElementID', 'Table', 'Path'])  # Write header
    for id, table, path in my_objects:
        path = os.path.normpath(path)
        csvwriter.writerow([id, table, path])

print("CSV file has been created successfully.")